In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install git+https://github.com/DataDog/dd-trace-py.git@yahya/update-prompt-annotation

  Cloning https://github.com/DataDog/dd-trace-py.git (to revision yahya/update-prompt-annotation) to /private/var/folders/fw/0099t8y529b2x_y472l_vnr40000gp/T/pip-req-build-0x5izpz9
  Running command git clone --filter=blob:none --quiet https://github.com/DataDog/dd-trace-py.git /private/var/folders/fw/0099t8y529b2x_y472l_vnr40000gp/T/pip-req-build-0x5izpz9
  Running command git checkout -b yahya/update-prompt-annotation --track origin/yahya/update-prompt-annotation
  Switched to a new branch 'yahya/update-prompt-annotation'
  branch 'yahya/update-prompt-annotation' set up to track 'origin/yahya/update-prompt-annotation'.
  Resolved https://github.com/DataDog/dd-trace-py.git to commit 6019952bd630716080fd2562ab86304287955dfd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [4]:
from ddtrace.llmobs import LLMObs
from ddtrace.llmobs.utils import Prompt
from ddtrace import patch
from openai import OpenAI
import openai
from dotenv import load_dotenv
import os

load_dotenv()
patch(openai=True)

openai.api_key = os.environ.get("openai_api_key")
client = OpenAI()
load_dotenv()

LLMObs.enable(
    api_key=os.environ.get("dd_api_key"),
    site=os.environ.get("dd_site"),
    ml_app="prompt-tracking-sandbox-v2",
    service="prompt-tracking-sandbox-v2",
    agentless_enabled=True,
    env="testing",
    integrations_enabled=False,
)

prompt_id = "prompt-tracking-sandbox-prompt"
prompt_name = "recommendation_engine"


In [5]:
def query_openai(system_prompt=None, user_prompt=None, variables=[], model="gpt-4o", max_tokens=100, temperature=0.7):
     """ Sends a prompt to the OpenAI API and returns the text response. """ 
     if system_prompt and user_prompt:
          system_prompt_rendered = system_prompt.format(**variables)
          user_prompt_rendered = user_prompt.format(**variables)
          completion = client.chat.completions.create(
               model=model,
               max_tokens=max_tokens,
               temperature=temperature,
               messages=[{
                    "role": "system",
                    "content": system_prompt_rendered
                    },
                    {
                    "role": "user",
                    "content": user_prompt_rendered
                    }]
          )
     elif user_prompt:
          user_prompt_rendered = user_prompt.format(**variables)
          completion = client.chat.completions.create(
               model=model,
               max_tokens=max_tokens,
               temperature=temperature,
               messages=[{
                    "role": "user",
                    "content": user_prompt_rendered
                    }]
          )
     elif system_prompt:
            system_prompt_rendered = system_prompt.format(**variables)
            completion = client.chat.completions.create(
                 model=model,
                 max_tokens=max_tokens,
                 temperature=temperature,
                 messages=[{
                        "role": "system",
                        "content": system_prompt_rendered
                        }]
            )
     return completion.choices[0].message.content

# How to annotate a prompt to a span 
In order to trace the prompt we wrap the openai call with LLMObs.prompt_context or LLMObs.annotation_context.
Example :

"""
with LLMObs.prompt_context(id=prompt_id, 
                           name=prompt_name, 
                           version="1.0.0",
                           template=[("User", prompt_step1)], 
                           variables=variables_step1
                           ):
"""
Or
"""
with LLMObs.annotation_context(prompt=Prompt(id=prompt_id,
                           name=prompt_name,
                           version="1.0.0",
                           template=[("User", prompt_step1)],
                           variables=variables_step1
                           )):
"""

# Step 1 simple user prompt

In [6]:
prompt_step1 = "{user_prompt}"
variables_step1 = {"user_prompt": "Suggest a destination and a restaurant."}

with LLMObs.annotation_context(prompt=Prompt(id=prompt_id, 
                           name=prompt_name, 
                           version="1.0.0",
                           template=[("User", prompt_step1)], 
                           variables=variables_step1
                           )):
    for i in range(10):
        response_step1 = query_openai(user_prompt=prompt_step1, variables=variables_step1)

print("=== USER Prompt Template (Step 1) ===") 
print(prompt_step1) 
print("\n=== Variables (Step 1) ===") 
print(variables_step1)
print("\n=== Response (Step 1) ===") 
print(response_step1)

=== USER Prompt Template (Step 1) ===
{user_prompt}

=== Variables (Step 1) ===
{'user_prompt': 'Suggest a destination and a restaurant.'}

=== Response (Step 1) ===
For a delightful travel experience, consider visiting Kyoto, Japan. Known for its stunning temples, traditional wooden houses, and beautiful gardens, Kyoto offers a serene and culturally rich atmosphere.

While in Kyoto, dine at "Kikunoi," a renowned kaiseki restaurant. Kikunoi offers an exquisite multi-course dining experience that showcases the best of Japanese seasonal ingredients and culinary artistry. The elegant presentation and harmonious flavors provide an unforgettable taste of Kyoto's culinary heritage. Be sure to make a reservation in advance to


# Step 2 add examples

In [7]:
prompt_step2 = """
    {user_prompt}
    Examples:
        {examples}
"""

# Examples
examples_step2 = """
Example 1:
Input: "Bob: Can you suggest a restaurant?"
Output: "Based on your preferences for Italian cuisine, I recommend trying Luigi's Trattoria."

Example 2:
Input: "Alice: What's a good place for nature walks?"
Output: "Considering your love for scenic views, I suggest visiting Green Park Gardens."
"""

variables_step2 = {
    "user_prompt": "Suggest a destination and a restaurant.", 
    "examples": examples_step2
    }

with LLMObs.annotation_context(prompt=Prompt(id=prompt_id, 
                           name=prompt_name, 
                           version="1.1.0",
                           template=[("User", prompt_step2)], 
                           variables=variables_step2
                           )):
    for i in range(10):
        response_step2 = query_openai(user_prompt=prompt_step2, variables=variables_step2)

print("=== USER Prompt template (Step 2) ===") 
print(prompt_step2) 
print("\n=== Variables (Step 2) ===")
print(variables_step2)
print("\n=== Response (Step 2) ===") 
print(response_step2)

=== USER Prompt template (Step 2) ===

    {user_prompt}
    Examples:
        {examples}


=== Variables (Step 2) ===
{'user_prompt': 'Suggest a destination and a restaurant.', 'examples': '\nExample 1:\nInput: "Bob: Can you suggest a restaurant?"\nOutput: "Based on your preferences for Italian cuisine, I recommend trying Luigi\'s Trattoria."\n\nExample 2:\nInput: "Alice: What\'s a good place for nature walks?"\nOutput: "Considering your love for scenic views, I suggest visiting Green Park Gardens."\n'}

=== Response (Step 2) ===
Input: "Sam: Where should I go for a cultural experience?"  
Output: "Given your interest in cultural experiences, I recommend visiting Kyoto, Japan, and dining at Kikunoi for an authentic kaiseki meal."


# Step 3 add system prompt with constraints and examples and lighten the user prompt

In [8]:
system_prompt_step3 = """
            You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences.
            Here are some rules and examples:

            - Always base your responses on user preferences.
            - Do not provide answers beyond the scope of the context given.

            Example exchanges:
            {examples}

            Use these examples to format your responses. 

            {constraints}

            Now, here's your specific task:
    """

prompt_step3 = "{user_prompt}"

examples_step3 = """
Example 1:
Input: "Bob: Can you suggest a restaurant?"
Output: "Based on your preferences for Italian cuisine, I recommend trying Luigi's Trattoria."

Example 2:
Input: "Alice: What's a good place for nature walks?"
Output: "Considering your love for scenic views, I suggest visiting Green Park Gardens."
"""

variables_step3 = {
    "user_prompt": "Suggest a destination and a restaurant.", 
    "examples": examples_step3, 
    "constraints": "Do not ask the user for more information. Either provide a recommendation or state that data is insufficient for a recommendation."
    }

with LLMObs.annotation_context(prompt=Prompt(id=prompt_id, 
                           name=prompt_name, 
                           version="2.0.0",
                           template=[("System",system_prompt_step3),("User", prompt_step3)], 
                           variables=variables_step3
                           )):
    for i in range(10):
        response_step3 = query_openai(system_prompt=system_prompt_step3, user_prompt=prompt_step3, variables=variables_step3)

print("=== SYSTEM Prompt template (Step 3) ===")
print(system_prompt_step3)
print("=== USER Prompt template (Step 3) ===") 
print(prompt_step3)
print("\n=== Variables (Step 3) ===")
print(variables_step3)
print("\n=== Response (Step 3) ===") 
print(response_step3)

=== SYSTEM Prompt template (Step 3) ===

            You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences.
            Here are some rules and examples:

            - Always base your responses on user preferences.
            - Do not provide answers beyond the scope of the context given.

            Example exchanges:
            {examples}

            Use these examples to format your responses. 

            {constraints}

            Now, here's your specific task:
    
=== USER Prompt template (Step 3) ===
{user_prompt}

=== Variables (Step 3) ===
{'user_prompt': 'Suggest a destination and a restaurant.', 'examples': '\nExample 1:\nInput: "Bob: Can you suggest a restaurant?"\nOutput: "Based on your preferences for Italian cuisine, I recommend trying Luigi\'s Trattoria."\n\nExample 2:\nInput: "Alice: What\'s a good place for nature walks?"\nOutput: "Considering your love for sceni

# Step 4 add context to system prompt and name of the user

In [9]:
system_prompt_step4 = """
        You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences.
        Here are some rules and examples:

        - Always base your responses on user preferences.
        - Do not provide answers beyond the scope of the context given.

        Example exchanges:
        {examples}

        Knowing the users' preferences is key to providing accurate recommendations. Here is the context for the current user:
        {name}'s Context: {context}

        Use these examples to format your responses. 

        {constraints}

        Now, here's your specific task:
    """

prompt_step4 = "{user_prompt}"

examples_step4 = """
Example 1:
Input: "Bob: Can you suggest a restaurant?"
Output: "Based on your preferences for Italian cuisine, I recommend trying Luigi's Trattoria."

Example 2:
Input: "Alice: What's a good place for nature walks?"
Output: "Considering your love for scenic views, I suggest visiting Green Park Gardens."
"""

variables_step4 = {
    "name": "Alice",
    "context": "Alice is allergic to peanuts and enjoys traveling to beaches and mountains. She prefers vegetarian food.",
    "user_prompt": "Suggest a destination and a restaurant.", 
    "examples": examples_step4, 
    "constraints": "Do not ask the user for more information. Either provide a recommendation or state that data is insufficient for a recommendation."
    }

with LLMObs.annotation_context(prompt=Prompt(id=prompt_id, 
                           name=prompt_name, 
                           version="2.1.0",
                           template=[("System",system_prompt_step4),("User", prompt_step4)], 
                           variables=variables_step4
                           )):
    for i in range(10):
        response_step4 = query_openai(system_prompt=system_prompt_step4, user_prompt=prompt_step4, variables=variables_step4)

print("=== SYSTEM Prompt template (Step 4) ===")
print(system_prompt_step4)
print("=== USER Prompt template (Step 4) ===") 
print(prompt_step4)
print("\n=== Variables (Step 4) ===")
print(variables_step4)
print("\n=== Response (Step 4) ===") 
print(response_step4)

=== SYSTEM Prompt template (Step 4) ===

        You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences.
        Here are some rules and examples:

        - Always base your responses on user preferences.
        - Do not provide answers beyond the scope of the context given.

        Example exchanges:
        {examples}

        Knowing the users' preferences is key to providing accurate recommendations. Here is the context for the current user:
        {name}'s Context: {context}

        Use these examples to format your responses. 

        {constraints}

        Now, here's your specific task:
    
=== USER Prompt template (Step 4) ===
{user_prompt}

=== Variables (Step 4) ===
{'name': 'Alice', 'context': 'Alice is allergic to peanuts and enjoys traveling to beaches and mountains. She prefers vegetarian food.', 'user_prompt': 'Suggest a destination and a restaurant.', 'examples': '\n

In [10]:
system_prompt_step5 = """
    You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences while maintaining the given context. Here are the rules:
    - Base recommendations primarily on user preferences.
    - Respect and adhere strictly to the context boundaries; do not go beyond the provided scope.
    - Personalization is key to every response.

    Knowing the users' preferences is key to providing accurate recommendations. Here is the context for the current user:
        {name}'s Context: {context}

    Example exchanges: {examples}

    Apply these examples to guide your response format, ensuring a focus on personal preferences within context. {constraints} 
    Now, here's your specific task:
    """

prompt_step5 = "{user_prompt}"

examples_step5 = """
Example 1:
Input: "Bob: Can you suggest a restaurant?"
Output: "Based on your preferences for Italian cuisine, I recommend trying Luigi's Trattoria."

Example 2:
Input: "Alice: What's a good place for nature walks?"
Output: "Considering your love for scenic views, I suggest visiting Green Park Gardens."
"""

variables_step5 = {
    "name": "Alice",
    "context": "Alice is allergic to peanuts and enjoys traveling to beaches and mountains. She prefers vegetarian food.",
    "user_prompt": "Suggest a destination and a restaurant.", 
    "examples": examples_step5, 
    "constraints": "Do not ask the user for more information. Either provide a recommendation or state that data is insufficient for a recommendation."
    }

with LLMObs.annotation_context(prompt=Prompt(id=prompt_id, 
                           name=prompt_name, 
                           version="2.2.0",
                           template=[("System",system_prompt_step5),("User", prompt_step5)], 
                           variables=variables_step5
                           )):
    for i in range(10):
        response_step5 = query_openai(system_prompt=system_prompt_step5, user_prompt=prompt_step5, variables=variables_step5)

print("=== SYSTEM Prompt template (Step 5) ===")
print(system_prompt_step5)
print("=== USER Prompt template (Step 5) ===") 
print(prompt_step5)
print("\n=== Variables (Step 5) ===")
print(variables_step5)
print("\n=== Response (Step 5) ===") 
print(response_step5)

=== SYSTEM Prompt template (Step 5) ===

    You are a recommendation bot with a focus on user preferences. Your main task is to provide the best recommendations based on provided user preferences while maintaining the given context. Here are the rules:
    - Base recommendations primarily on user preferences.
    - Respect and adhere strictly to the context boundaries; do not go beyond the provided scope.
    - Personalization is key to every response.

    Knowing the users' preferences is key to providing accurate recommendations. Here is the context for the current user:
        {name}'s Context: {context}

    Example exchanges: {examples}

    Apply these examples to guide your response format, ensuring a focus on personal preferences within context. {constraints} 
    Now, here's your specific task:
    
=== USER Prompt template (Step 5) ===
{user_prompt}

=== Variables (Step 5) ===
{'name': 'Alice', 'context': 'Alice is allergic to peanuts and enjoys traveling to beaches and mo